In [1]:
from sklearn import datasets
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import queue

In [10]:
class bt:
    def __init__(self,entrophy,level,split_feature,gain):
        self.entrophy=entrophy
        self.level=level
        self.split_feature=split_feature
        self.gain=gain
        self.left=None
        self.right=None
    
def printbt1(root,s):
        if root is None:
            return
        print(s)
        print("level:-",root.level)
        print("entrophy:-",root.entrophy)
        print("split_feature:-",root.split_feature)
        print("Gain:-",root.gain)
        print()
        
        printbt1(root.left,"Left Node")
        printbt1(root.right,"Right Node")
    

In [3]:
iris=datasets.load_iris()
x=pd.DataFrame(iris.data)
y=pd.DataFrame(iris.target)
x.columns=['sl','sw','pl','pw']
features=['sl','sw','pl','pw']
level=0
y.columns=['flower_type']
# y
# iris

In [4]:
def countsetosa(df):
    out=np.array(df[:])
    return (out==0).sum()
def countversicolor(df):
    out=np.asarray(df[:])
    return (out==1).sum()


In [5]:
def entrophy(lst):
    info=0
    if sum(lst)==0:
        return 0
    for i in range(3):
        if lst[i]/sum(lst)==0:
            continue
        info+= ((-1)*(lst[i]/sum(lst)))*math.log(lst[i]/sum(lst),2)
    return info

In [6]:
def gain_ratio(lst,lst1,lst2):
    info=entrophy(lst)
    info1=entrophy(lst1)
    info2=entrophy(lst2)
    
    a=(sum(lst1)/(sum(lst1)+sum(lst2)))*info1
    b=(sum(lst2)/(sum(lst1)+sum(lst2)))*info2
    info_gain=info-a-b
    
    if (sum(lst1)/(sum(lst1)+sum(lst2)))==0:
        split1=0
    else:
        split1=-1*(sum(lst1)/(sum(lst1)+sum(lst2)))*math.log((sum(lst1)/(sum(lst1)+sum(lst2))),2)
    if (sum(lst2)/(sum(lst1)+sum(lst2)))==0:
        split2=0
    else:
        split2=-1*(sum(lst2)/(sum(lst1)+sum(lst2)))*math.log((sum(lst2)/(sum(lst1)+sum(lst2))),2)
    split_info=split1+split2
    try:
        gain_rati=info_gain/split_info
    except:
        gain_rati=0
        
    return gain_rati

In [7]:
def gain(x,y,f):
    data=x[f]
    data=np.array(data)
    maxx=0
    feat=None
    mid=0
    
    for i in range(1,len(data)):
        m=(data[i-1]+data[i])/2
        lst=[0,0,0]
        lst1=[0,0,0]
        lst2=[0,0,0]
        
        split1x=x[data>m]
        split1y=y[data>m]
        
        split2x=x[data<=m]
        split2y=y[data<=m]
        
        total_element=len(x)
        lst[0]=countsetosa(y)
        lst[1]=countversicolor(y)
        lst[2]=total_element-lst[0]-lst[1]
        
        total_ele1=len(split1x)
        lst1[0]=countsetosa(split1y)
        lst1[1]=countversicolor(split1y)
        lst1[2]=total_ele1-lst1[0]-lst1[1]
        
        total_ele2=len(split2x)
        lst2[0]=countsetosa(split2y)
        lst2[1]=countversicolor(split2y)
        lst2[2]=total_ele2-lst2[0]-lst2[1]
        
        if lst1.count(0)==3 and lst2.count(0)==3:
            continue
        
        gr=gain_ratio(lst,lst1,lst2)
        if gr>=maxx:
            maxx=gr
            feat=f
            mid=m
    return maxx,feat,mid 

In [8]:
def dt(x,y,feature,level):
    lst=[0,0,0]
    total_ele=len(x)
    lst[0]=countsetosa(y)
    lst[1]=countversicolor(y)
    lst[2]=total_ele-countsetosa(y)-countversicolor(y)
    
    print("level",level)
   
    print("number of setosa:-",countsetosa(y))
    print("number of versicolor:-",countversicolor(y))
    print("number of virginica:-",total_ele-countversicolor(y)-countsetosa(y))
    print("current entrophy:-",entrophy(lst))
    
    if(lst.count(0)==2):
        root=bt(entrophy(lst),level,"Leaf Node reached",0)
        return root
    mid=0
    best_feat=None
    maxx=0
    
    for f in feature:
        gr,fe,m=gain(x,y,f)
        if gr>=maxx:
            maxx=gr
            best_feat=fe
            mid=m
    print("spliting on feature ",best_feat,"with gain ratio",maxx)
    root=bt(entrophy(lst),level,"with gain ratio",maxx)
    split1x=x[x[best_feat]>mid]
    split1y=y[x[best_feat]>mid]
    
    split2x=x[x[best_feat]<=mid]
    split2y=y[x[best_feat]<=mid]
    
    
    
    root.left=dt(split1x,split1y,feature,level+1)
    root.right=dt(split2x,split2y,feature,level+1)
    return root

root=dt(x,y,features,level)
print("=========================================")

level 0
number of setosa:- 50
number of versicolor:- 50
number of virginica:- 50
current entrophy:- 1.584962500721156
spliting on feature  pw with gain ratio 0.9999999999999999
level 1
number of setosa:- 0
number of versicolor:- 50
number of virginica:- 50
current entrophy:- 1.0
spliting on feature  pw with gain ratio 0.6933647985912662
level 2
number of setosa:- 0
number of versicolor:- 1
number of virginica:- 45
current entrophy:- 0.15109697051711368
spliting on feature  pl with gain ratio 0.2622302372762406
level 3
number of setosa:- 0
number of versicolor:- 0
number of virginica:- 43
current entrophy:- 0.0
level 3
number of setosa:- 0
number of versicolor:- 1
number of virginica:- 2
current entrophy:- 0.9182958340544896
spliting on feature  sw with gain ratio 1.0
level 4
number of setosa:- 0
number of versicolor:- 1
number of virginica:- 0
current entrophy:- 0.0
level 4
number of setosa:- 0
number of versicolor:- 0
number of virginica:- 2
current entrophy:- 0.0
level 2
number of se

C:\Users\Dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


spliting on feature  pl with gain ratio 0.6066178220203009
level 3
number of setosa:- 0
number of versicolor:- 0
number of virginica:- 2
current entrophy:- 0.0
level 3
number of setosa:- 0
number of versicolor:- 49
number of virginica:- 3
current entrophy:- 0.31821529768323314
spliting on feature  pl with gain ratio 0.2720453440631924
level 4
number of setosa:- 0
number of versicolor:- 2
number of virginica:- 2
current entrophy:- 1.0
spliting on feature  pw with gain ratio 1.0
level 5
number of setosa:- 0
number of versicolor:- 2
number of virginica:- 0
current entrophy:- 0.0
level 5
number of setosa:- 0
number of versicolor:- 0
number of virginica:- 2
current entrophy:- 0.0
level 4
number of setosa:- 0
number of versicolor:- 47
number of virginica:- 1
current entrophy:- 0.14609425012013633
spliting on feature  pw with gain ratio 0.26298064861912657
level 5
number of setosa:- 0
number of versicolor:- 2
number of virginica:- 1
current entrophy:- 0.9182958340544896
spliting on feature  p

In [11]:
printbt1(root,"Root_Node")

Root_Node
level:- 0
entrophy:- 1.584962500721156
split_feature:- with gain ratio
Gain:- 0.9999999999999999

Left Node
level:- 1
entrophy:- 1.0
split_feature:- with gain ratio
Gain:- 0.6933647985912662

Left Node
level:- 2
entrophy:- 0.15109697051711368
split_feature:- with gain ratio
Gain:- 0.2622302372762406

Left Node
level:- 3
entrophy:- 0.0
split_feature:- Leaf Node reached
Gain:- 0

Right Node
level:- 3
entrophy:- 0.9182958340544896
split_feature:- with gain ratio
Gain:- 1.0

Left Node
level:- 4
entrophy:- 0.0
split_feature:- Leaf Node reached
Gain:- 0

Right Node
level:- 4
entrophy:- 0.0
split_feature:- Leaf Node reached
Gain:- 0

Right Node
level:- 2
entrophy:- 0.44506485705083865
split_feature:- with gain ratio
Gain:- 0.6066178220203009

Left Node
level:- 3
entrophy:- 0.0
split_feature:- Leaf Node reached
Gain:- 0

Right Node
level:- 3
entrophy:- 0.31821529768323314
split_feature:- with gain ratio
Gain:- 0.2720453440631924

Left Node
level:- 4
entrophy:- 1.0
split_feature:- wit